In [47]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import multiprocessing as mp
from multiprocessing import Pool, Value, Array, Process, current_process
import time
import os
import numpy as np
import random

In [48]:
def read_4bytes():
    numbers = []
    with open('example', 'br') as f:
        data = f.read(4)
        while data:
            number = int.from_bytes(data, "big", signed=True)
            numbers.append(number)
            data = f.read(4)
    return numbers


In [49]:
def count_frequency(x): 
    freq = {} 
    for item in x: 
        if (item in freq): 
            freq[item] += 1
        else: 
            freq[item] = 1
    return freq

In [50]:
def random_choices(x, tries):
    sequence = []
    while len(sequence) < tries:
        sequence.append(random.choice(x))
    return sequence

In [51]:
def make_subsample(num_items, x):
    resulting_sequence = []
    weights = []
    while num_items:
        number_of_tries = 1000000
        sequence = random_choices(x, number_of_tries)
        freq = count_frequency(sequence)
        highest = max(freq.values())
        for key, value in freq.items():
            if value == highest:
                resulting_sequence.append(key)
                weights.append(value / number_of_tries)
                x.remove(key)
                break ;
        random.shuffle(x)
        num_items -= 1
    return resulting_sequence

In [52]:
def check_the_pid(subsample):
    subsample = make_subsample(N, x)
    proc_file = str(current_process().name) + ".txt"
    pid = os.getpid()
    f = open(proc_file, "a+")
    res = []
    for i in subsample:
        if i % (3 * pid) == 0:
            res.append(i)
            f.write("%d\r\n" % (i))
    f.close()
    return res

In [ ]:
def check_the_pid_(args):
    return check_the_pid(*args)

In [53]:
#%%time
#def test():
#    N = 100
#    x  = read_4bytes()
#    subsample = make_subsample(N, x)
#    print(subsample)
#    print(check_the_pid(subsample))
#test()

In [54]:
def multiprocessing_pool():
    PROCESSES = 4
    N = 100
    x  = read_4bytes()
    with ProcessPoolExecutor(max_workers = PROCESSES) as pool:
        #params = read_and_get()
        results = pool.map(check_the_pid_, np.array_split(subsample, N), chunksize=100)
    return results

In [55]:
%%time
PROCESSES = 4
N = 100
x  = read_4bytes()

CPU times: user 6.42 ms, sys: 0 ns, total: 6.42 ms
Wall time: 5.95 ms


In [56]:
%%time

multiprocessing_pool()

CPU times: user 1min 24s, sys: 91.8 ms, total: 1min 24s
Wall time: 1min 24s


<generator object _chain_from_iterable_of_lists at 0x7ff5e9ad7550>